In [1]:
#first let us import the necessary modules

import matplotlib.pyplot as plt
from preprocessing import *
from ispy_utils import *
import numpy as np

In [2]:
#setting file paths

#precontrast image path
file_pre ='ispy_data/in/manifest-1724022411242/ISPY2/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/500.000000-ISPY2 AX VIBRANT-SERIES-52523'

#postcontrast image path
file_post = 'ispy_data/in/manifest-1724022411242/ISPY2/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/501.000000-ISPY2 Ph1AX VIBRANT-SERIES-03969'



In [3]:
#apply function from ispymri.py 
image_array_pre, dcm_data_pre = read_precontrast_mri_ispy(file_pre)
image_array_pre = zscore_image(normalize_image(image_array_pre))
# saving image - will be used as an input to prediction code
np.save('./input_pre/output.npy', image_array_pre)
print(image_array_pre.shape)  #optional

image_array_post, dcm_data_post = read_precontrast_mri_ispy(file_post)
image_array_post = zscore_image(normalize_image(image_array_post))
np.save('./inputpost/output.npy', image_array_post)

NameError: name 'read_precontrast_mri_ispy' is not defined

In [39]:
import matplotlib.pyplot as plt
#from preprocessing import *
from ispy_utils import *

 
#location of downloaded pre-contrast dicom file
file_pre ='ispy_data/in/manifest-1724022411242/ISPY2/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/500.000000-ISPY2 AX VIBRANT-SERIES-52523'

image_array_pre, dcm_data_pre = read_precontrast_mri_ispy(file_pre)
image_array_pre = zscore_image(normalize_image(image_array_pre))
#The saved image array shall be used as an input to the prediction code to generate the breast segmentation mask
np.save('./inputpre/output.npy', image_array_pre)
print(image_array_pre.shape)

plt.subplot(1, 2, 1)
plt.title('MRI Pre Volume')
plt.imshow(image_array_pre[:, :, 41], cmap='gray')
plt.axis('off')

#location of downloaded post-contrast dicom file
file_post = 'ispy_data/in/manifest-1724022411242/ISPY2/ISPY2-100899/10-26-2002-100899T0-ISPY2MRIT0-88595/501.000000-ISPY2 Ph1AX VIBRANT-SERIES-03969'
image_array_post, dcm_data_post = read_precontrast_mri_ispy(file_post)
image_array_post = zscore_image(normalize_image(image_array_post))
np.save('./inputpost/output.npy', image_array_post)


plt.subplot(1, 2, 2)
plt.title('MRI Post Volume')
plt.imshow(image_array_post[:, :, 41], cmap='gray')
plt.axis('off')

NameError: name 'read_precontrast_mri_ispy' is not defined